### Init

In [ ]:
#import  RNAseqQueryingInit
##static params
baseDir='/home/jovyan/efs/all_seq/microbes/' #Base directory

import warnings
warnings.filterwarnings('ignore')
import ipywidgets as widgets
import os
import pandas as pd
from IPython.display import display,Javascript
from ipywidgets import  Layout


def run_all(ev):
    display(Javascript('IPython.notebook.execute_cells_below()'))

exampleQuery='HeLa Cells' 
style = {'description_width': 'initial'}
widget_query=widgets.Text(
    #value='',
    placeholder='Enter conditions seperated by comma to search and compare: eg. B-Cell,T-Cell',
    description='',
    disabled=False,
    #description='(50% width, 80px height) button',
    layout=widgets.Layout(width='80%', height='50px'),
    style=style
    
)

baseDir_FnameS=pd.Series(os.listdir(baseDir))
speciesWithReprocessedData=baseDir_FnameS[baseDir_FnameS.str.contains('.npy$')].str.split('.').str[0].unique()
"""
widget_specie=widgets.Select(
    options=speciesWithReprocessedData,
    value='Homo_sapiens',
    # rows=10,
    description='Select your species:',
    disabled=False,
    style=style
)"""

button_query = widgets.Button(description="Search",
                             layout=Layout(width='20%', height='10%'))
button_query.on_click(run_all)
button_query.style.button_color='lightblue'
accordion = widgets.HBox(children=[widget_query,button_query])

checkbox_exportCSV=widgets.Checkbox(
    value=False,
    description='Export output matrix as CSV',
    disabled=False
)


# Search and compare RNA-seq profiles based on experimental conditions

**Refresh page if you do not see the ipywidgets from below**

In [ ]:
bacterialAnnotationLevel=2 #feel free to increase the bacterial clade resolution
display(accordion)
#display(widget_specie)
display(checkbox_exportCSV)

<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>



# Execution

### More parameters: 

expression_metric: Those are Kallisto expression metric: "tpm","est_counts"
baseDir: if run locally, change it to mirror our path.

In [ ]:
#from  RNAseqQueryingInit import *
import re

In [ ]:
expression_metric='tpm' #

In [ ]:
#querySpecie=widget_specie.get_interact_value()

In [ ]:
queryStr=widget_query.get_interact_value()

listOfQueries=re.split(" *, *", queryStr)

if len(queryStr)<3:
    raise ValueError('Please provide a query with more than 3 characters')
#if len(listOfQueries)<2:
#    raise ValueError('Please provide a query with more than 2 conditions')


In [ ]:
queryLabelToRegexDict=dict(zip(listOfQueries,listOfQueries))

## Data loading

### load in SRS biospecieman annotations

In [ ]:
%matplotlib notebook

import pandas as pd
import numpy as np

allSRS_pickle_dir='/home/jovyan/efs/all_seq/meta_data/allSRS.with_processed_data.flat.pickle.gz'
%time allSRS=pd.read_pickle(allSRS_pickle_dir)
allSRS.index.names=['SRS']

### load in technical metadata

In [ ]:
sra_dump_pickle_dir='/home/jovyan/efs/all_seq/meta_data/sra_dump.fastqc.bowtie_algn.pickle'
%time technical_meta_data_df=pd.read_pickle(sra_dump_pickle_dir)
technical_meta_data_df[('SRAmeta','Run')]=technical_meta_data_df.index

### load the expression matrix

Check files in baseDir directory for more species

In [ ]:
def loadDf(fname,mmap_mode='r'):
    with open(fname+'.index.txt') as f:
        myIndex=map(lambda s:s.replace("\n",""), f.readlines())
    with open(fname+'.columns.txt') as f:
        myColumns=map(lambda s:s.replace("\n",""), f.readlines())
    tmpMatrix=np.load(fname+".npy",mmap_mode=mmap_mode)
    tmpDf=pd.DataFrame(tmpMatrix,index=myIndex,columns=myColumns)
    tmpDf.columns.name='Run'
    return tmpDf
data_matrix_dir=baseDir+'microbes'

%time rnaseqDf=loadDf(data_matrix_dir)

In [ ]:
#!ls $baseDir

### Find the relevent SRS (Sample  IDs)  


In [ ]:
myL=[]
for  queryRegex in queryLabelToRegexDict.values():
    %time hitSrsS=allSRS[allSRS.str.contains(queryRegex,case=False)]
    myL.append(hitSrsS)

queryLabel='queryLabel'
mergeS=pd.concat(myL,keys=queryLabelToRegexDict.keys(),names=[queryLabel])
mergeS_noDup=mergeS.groupby(['SRS','queryLabel']).first()
unqiueHitMask=mergeS_noDup.groupby('SRS').size()==1
unqiueHitSrs=unqiueHitMask.index[unqiueHitMask]
mergeS_noDup_unique=mergeS_noDup[mergeS_noDup.index.get_level_values('SRS').isin(unqiueHitSrs)]

Number of SRS per query class

In [ ]:
mergeS_noDup_unique.groupby(queryLabel).size()

In [ ]:
srsToClasses_all=mergeS_noDup_unique.reset_index().set_index(['SRS'])['queryLabel']

srsToClasses=srsToClasses_all

### map SRS Ids to SRR Ids

In [ ]:
m_SRAMeta=technical_meta_data_df[('SRAmeta','Sample')].isin(srsToClasses.index)
technical_meta_data_df_hit=technical_meta_data_df[m_SRAMeta]

SRAMetasrsCorrespondingQuery=srsToClasses.loc[technical_meta_data_df_hit[('SRAmeta','Sample')]].values
technical_meta_data_df_hit[('SRAmeta',queryLabel)]=SRAMetasrsCorrespondingQuery
relevantMetaColsL=[('SRAmeta',queryLabel),('SRAmeta','Study'),('SRAmeta','Sample'),('SRAmeta','Run'),('SRAmeta','ScientificName')]
technical_meta_data_df_sub=technical_meta_data_df_hit[relevantMetaColsL]
designDf=technical_meta_data_df_sub['SRAmeta']

Top species with # of reprocessed profiles

In [ ]:
print ('# expression profiles per query class and species available in SRA: ',designDf.groupby(['queryLabel','ScientificName']).size())

In [ ]:
hitSrsAllAnnotS=allSRS[allSRS.index.get_level_values('SRS').isin(mergeS.index.get_level_values('SRS'))]

In [ ]:
srsToTextS=hitSrsAllAnnotS

In [ ]:
srsToTextS=pd.Series(data="NCBI SRA SRS:"+srsToTextS.index+' <br> '+srsToTextS.values,index=srsToTextS.index)

In [ ]:
designDf['Description']=srsToTextS[designDf.Sample].values

### Subset the set of reprocessed data

In [ ]:
%time designDf_specie=designDf[(designDf.Run.isin(rnaseqDf.columns))]
queryDesignDf=designDf_specie

In [ ]:
print ('Number of samples per query class that have data reprocessed in SkyMap: ',designDf_specie.groupby(queryLabel).size())


In [ ]:
#rnaseqDf

In [ ]:
from tqdm import tqdm

In [ ]:
%time hitDf=pd.DataFrame( list(tqdm (map( lambda srrId: rnaseqDf[srrId],queryDesignDf.Run)))).T


In [ ]:
hitDf.columns=queryDesignDf.set_index(queryDesignDf.columns.tolist()).index

In [ ]:
bacterialAnnotationDf=pd.read_pickle('../efs/all_seq/microbes/bacterial.annot.pickle')
viralAnnotationS=pd.read_pickle('../efs/all_seq/microbes/viral.annot.pickle')

In [ ]:
renameS=pd.concat([bacterialAnnotationDf[bacterialAnnotationLevel],viralAnnotationS])

In [ ]:
renameS=renameS.groupby(renameS.index).first()

In [ ]:
hitDf.index=renameS[hitDf.index].values

In [ ]:
hitDf_summed=hitDf.sum(axis=0,level=[0])

# Export and download the expression matrix


### Output: fully annotated matrix matrix

The quantification metric is TPM by default. Expand and change the parameters in code section "Execution" if you want raw counts instead. 

Example layout is listed in the cell below

In [ ]:
#hitDf

Export the returned dataframe for analysis

In [ ]:
hitDf_summed.head()

In [ ]:
m_with_microbe_read=(hitDf_summed.sum(axis=0)>0)
m_with_microbe_read.mean()

In [ ]:
hitDf_summed_withData=hitDf_summed.loc[:,m_with_microbe_read]

In [ ]:
hitDf_summed_withData.to_pickle('./tmp.profile.pickle')


Hit the following button to download the expression profile matrix. 
If you see a dialogue box when you hit the following download button, click "Leave page"

In [ ]:
if checkbox_exportCSV.get_interact_value():
    hitDf.to_csv('./tmp.csv')
    from IPython.core.display import display, HTML
    display(HTML('<form method="get" action="tmp.csv">\
       <button type="submit">Click here to download the profile matrix as an CSV file</button>\
    </form>'))

# Click here to view the analysis associated with the returned matrix
**Please make sure you have run "search" before running the following analysis:**
* [Volcano plot with t-test (Works only when there are more than 2 conditions)](VolcanoPlot.ipynb)
* [Boxplot showing the read counts levels of a microbe sequence ](QueryGenesInConditions.ipynb)
* [Interactive PCA 2D/ 3D ](./RNAseqPCA.ipynb)
* [Interactive TSNE](./RNAseqTSNE.ipynb)


The raw counts probably gonna suck hard for for PCA and TSNE analysis. It's preferable that you use the microbe data caustiously. 